# Exercice 1 : Construire l’index simple

In [2]:
import os
import glob
import json
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /home/priscille/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Importation de Dossier de corpus

In [3]:
corpus_path = "./europarl/fr/"

In [ ]:
def lire_fichier(path):
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        return f.read()

def decouper_mots(texte):
    return texte.lower().split()

index = {}

fichiers = glob.glob(os.path.join(corpus_path, "*"))[:5]

for chemin in fichiers:
    contenu = lire_fichier(chemin)
    mots = decouper_mots(contenu)
    for mot in mots:
        if mot not in index:
            index[mot] = set()
        index[mot].add(chemin)

print("Taille du vocabulaire :", len(index))
print("Quelques mots indexés :", list(index.keys())[:50])

Taille du vocabulaire : 23493
Quelques mots indexés : ['reprise', 'de', 'la', 'session', 'je', 'déclare', 'du', 'parlement', 'européen', 'qui', 'avait', 'été', 'interrompue', 'le', 'jeudi', '30', 'mai', '2002.', 'comme', 'vous', 'savez,', 'chers', 'collègues,', 'principe', 'transparence', 'est', 'consacré', 'dans', "l'article", 'premier', 'traité', 'sur', "l'union", 'européenne.', 'son', 'entrée', 'en', 'vigueur', 'a', 'annoncé', 'une', 'nouvelle', 'ère', 'laquelle', 'les', 'décisions', 'doivent', 'se', 'prendre', 'un']


 Exemple : documents contenant le mot "indique"

In [ ]:
mot_exemple = "indique"
if mot_exemple in index:
    print(f"Documents contenant '{mot_exemple}' :", index[mot_exemple])
else:
    print(f"Mot '{mot_exemple}' non trouvé dans l'index.")

index_serializable = {mot: list(fichiers) for mot, fichiers in index.items()}
with open("index.json", "w", encoding="utf-8") as f:
    json.dump(index_serializable, f)

print("Index sauvegardé dans 'index.json'.")

Documents contenant 'indique' : {'./europarl/fr/ep-03-06-18-fr.txt', './europarl/fr/ep-03-03-12-fr.txt'}
Index sauvegardé dans 'index.json'.


exemple 2 : affichage de nombre de textes contenant certains mots

In [ ]:
def compter_documents(mot, index):
    mot = mot.lower()
    if mot in index:
        return len(index[mot])
    else:
        return 0

mots_a_verifier = ["indique", "européenne", "toto"]

for mot in mots_a_verifier:
    nombre_textes = compter_documents(mot, index)
    print(f"Le mot '{mot}' apparaît dans {nombre_textes} document(s).")

index_serializable = {mot: list(paths) for mot, paths in index.items()}
with open("index.json", "w", encoding="utf-8") as f:
    json.dump(index_serializable, f)

print("Index sauvegardé dans 'index.json'.")

Le mot 'indique' apparaît dans 2 document(s).
Le mot 'européenne' apparaît dans 5 document(s).
Le mot 'toto' apparaît dans 0 document(s).
Index sauvegardé dans 'index.json'.


 # Exercice 2 : Requêter le corpus
 Objectif :

    - Charger l’index sauvegardé (index.json)
    - Découper une requête en mots
    - Trouver tous les documents qui contiennent au moins un des mots
    - Améliorer : éviter doublons, limiter à 10 documents, trier selon le nombre de mots présents



Charger l'index sauvegardé

In [12]:
import nltk
nltk.download('punkt')
nltk.download('perluniprops')
nltk.download('nonbreaking_prefixes')


[nltk_data] Downloading package punkt to /home/priscille/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/priscille/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/priscille/nltk_data...
[nltk_data]   Unzipping corpora/nonbreaking_prefixes.zip.


True

In [13]:

with open("index.json", "r", encoding="utf-8") as f:
    index = json.load(f)
print("Index chargé avec succès.")

Index chargé avec succès.


In [ ]:
from nltk.tokenize import wordpunct_tokenize

def decouper_requete(requete):
    return wordpunct_tokenize(requete.lower())

def chercher_documents(requete, index):
    mots_requete = decouper_requete(requete)
    documents_trouves = {}

    for mot in mots_requete:
        if mot in index:
            for doc in index[mot]:
                if doc not in documents_trouves:
                    documents_trouves[doc] = 0
                documents_trouves[doc] += 1 
    return documents_trouves

def afficher_documents(documents_trouves):
    if not documents_trouves:
        print("Aucun mot de la requête n'a été trouvé dans les documents.")
        return
    liste_triee = sorted(documents_trouves.items(), key=lambda x: x[1], reverse=True)

    print(f"Top {min(10, len(liste_triee))} documents les plus pertinents :")
    for doc, score in liste_triee[:10]:  
        print(f"Document : {doc.split('/')[-1]} | Score (mots trouvés) : {score}")

#Exemple d'utilisation

In [21]:
requete = "Commission Européenne"
documents_trouves = chercher_documents(requete, index)
afficher_documents(documents_trouves)

Top 5 documents les plus pertinents :
Document : ep-02-06-10-fr.txt | Score (mots trouvés) : 2
Document : ep-03-03-12-fr.txt | Score (mots trouvés) : 2
Document : ep-03-06-18-fr.txt | Score (mots trouvés) : 2
Document : ep-02-12-04-fr.txt | Score (mots trouvés) : 2
Document : ep-00-02-18-fr.txt | Score (mots trouvés) : 2


 # Exercice 3 : Afficher les contextes
 Quand on trouve un mot dans un document, on veut afficher le contexte autour du mot (avant/après), pas seulement le nom du fichier.


#Charger l'index sauvegardé

In [ ]:
import re

def afficher_contextes(texte, mot_recherche, taille_contexte=30):
    match = re.search(mot_recherche, texte, re.IGNORECASE) 
    contexts = []
    while match is not None:
        gauche = max(match.start() - taille_contexte, 0)
        droite = match.end() + taille_contexte
        contexts.append(texte[gauche:droite])
        texte = texte[match.end():]
        match = re.search(mot_recherche, texte, re.IGNORECASE)
    
    for c in contexts:
        print("...", c.strip(), "...")


Exemple d’utilisation : on a trouvé un document qui contient "Commission".
On veut afficher le contexte autour du mot dans ce fichier.

In [ ]:
if documents_trouves:
    premier_document = list(documents_trouves.keys())[0]
    texte_du_document = lire_fichier(premier_document)
    mot = "Commission"

    afficher_contextes(texte_du_document, mot, taille_contexte=50)
else:
    print("Aucun document trouvé pour afficher le contexte.")


... uments du Parlement européen, du Conseil et de la Commission.
        Le règlement exigeait des institutions q ...
... nt du groupe ELDR pour que la communication de la Commission sur le Livre vert sur la protection des consommat ...
... t une bonne chose.
      
      
        Quand la Commission a-t-elle eu connaissance de cette affaire ? ...
... s a-t-elle prises et quelle est la position de la Commission actuellement ?
        L'Europe dispose par aille ...
... ire de Dublin, placé sous la responsabilité de la Commission.
        Cet organe contrôle la totalité de la pr ...
... Europe.
      
      
    
    
      
        La Commission a fait savoir qu'elle serait heureuse d'aborder c ...
... 'invasion.
        Nous devons l'examiner avec la Commission et le Conseil, en nous rappelant que les quinze É ...
... 'un examen et de déclarations du Conseil et de la Commission, lors de la prochaine séance plénière du mois de ...
... édure de levée d'immunité se trouve au sein de la c

 On visualise l'affichage de plusieurs morceaux de texte autour du mot trouvé.

# Exercice 4 : Améliorer la tokenisation avec NLTK

Objectif :

    - Remplacer le découpage simple (split) par une vraie tokenisation linguistique avec nltk.word_tokenize.
    - Puis comparer la qualité (plus propre, meilleure séparation des mots, gestion de la ponctuation…).

In [29]:
from nltk.tokenize import wordpunct_tokenize

def decouper_mots(texte):
    return wordpunct_tokenize(texte.lower())



On refais l'index  avec le nouveau tokenizer NLTK

In [ ]:

index = {}

fichiers = glob.glob(os.path.join(corpus_path, "*"))

for chemin in fichiers:
    contenu = lire_fichier(chemin)
    mots = decouper_mots(contenu)
    for mot in mots:
        if mot not in index:
            index[mot] = set()
        index[mot].add(chemin)
index_serializable = {mot: list(paths) for mot, paths in index.items()}

with open("index.json", "w", encoding="utf-8") as f:
    json.dump(index_serializable, f)

print("Index reconstruit et sauvegardé avec une meilleure tokenisation (nltk.wordpunct_tokenize).")
print("Nouveau vocabulaire :", len(index_serializable), "mots.")


# Exercice 5 : Construire l’index et l’index inversé
Objectif:
  - Construire l’index inversé (document → liste de mots présents dans ce document)

In [33]:
def creer_index_inverse(index):
    index_inverse = {}
    for mot, documents in index.items():
        for doc in documents:
            if doc not in index_inverse:
                index_inverse[doc] = set()
            index_inverse[doc].add(mot)
    return index_inverse

index_inverse = creer_index_inverse(index)

print("Nombre de termes différents :", len(index.keys()))
print("Nombre de documents :", len(index_inverse.keys()))


Nombre de termes différents : 82144
Nombre de documents : 363


La sortie, on a l'index normal (mot → docs) et l'index inversé (doc → mots)

#  Exercice 6 : Calculer les TF et IDF
bjectif :

    - Calculer le TF (Term Frequency) de chaque mot dans chaque document.
    - Calculer le IDF (Inverse Document Frequency) pour chaque mot du corpus.
    - Les préparer pour utiliser plus tard dans la pondération TF-IDF et la similarité cosinus.

TF(mot, doc) = nombre d'occurrences du mot dans le document / nombre total de mots dans le document
IDF(mot) = log10 (nombre total de documents / nombre de documents contenant le mot)



In [ ]:
import math

# 1. Calculer les TF pour chaque document
def calculer_tf(index_inverse, corpus_path):
    all_docs_tf = {}
    
    for doc, mots in index_inverse.items():
        texte = lire_fichier(doc)
        mots_doc = decouper_mots(texte)
        tf_doc = {}
        total_mots = len(mots_doc)
        for mot in mots_doc:
            mot = mot.lower()
            if mot not in tf_doc:
                tf_doc[mot] = 0
            tf_doc[mot] += 1
        for mot in tf_doc:
            tf_doc[mot] /= total_mots
        all_docs_tf[doc] = tf_doc
    
    return all_docs_tf

# 2. Calculer les IDF pour chaque mot
def calculer_idf(index, total_documents):
    idf = {}
    for mot, docs in index.items():
        idf[mot] = math.log10(total_documents / len(docs)) if len(docs) != 0 else 0
    return idf
all_docs_tf = calculer_tf(index_inverse, corpus_path)
idf = calculer_idf(index, len(index_inverse))

print("TF et IDF calculés avec succès.")


TF et IDF calculés avec succès.


exemple 

In [35]:
print("TF d'un document :", list(all_docs_tf.items())[0])
print("IDF d'un mot :", list(idf.items())[0])


TF d'un document : ('./europarl/fr/ep-02-06-10-fr.txt', {'reprise': 6.381077976772877e-05, 'de': 0.041881141787552645, 'la': 0.02537542008763347, 'session': 0.0003615944186837963, 'je': 0.008082698770578977, 'déclare': 6.381077976772877e-05, 'du': 0.008444293189262772, 'parlement': 0.003594673926915387, 'européen': 0.0014889181945803378, 'qui': 0.008359212149572469, 'avait': 0.00038286467860637255, 'été': 0.0021270259922576253, 'interrompue': 2.1270259922576255e-05, 'le': 0.019547368868847576, 'jeudi': 0.00010635129961288127, '30': 6.381077976772877e-05, 'mai': 4.254051984515251e-05, '2002': 0.00021270259922576254, '.': 0.03196920066363211, 'comme': 0.001850512613264134, 'vous': 0.0020632152124898965, 'savez': 8.508103969030502e-05, ',': 0.04409324881950057, 'chers': 0.0003190538988386438, 'collègues': 0.0008295401369804739, 'principe': 0.00029778363891606755, 'transparence': 0.00029778363891606755, 'est': 0.008827157867869145, 'consacré': 4.254051984515251e-05, 'dans': 0.0071255370740

# Exercice 7 : Calcul du taux de similarité cosinus
Objectif :

    - Représenter chaque document et chaque requête comme un vecteur pondéré TF-IDF.
    - Calculer la similarité cosinus entre la requête et chaque document.
    - Le cosinus donne un score entre 0 et 1 :
        -  1 = document parfaitement similaire
        - 0 = document totalement différent

In [ ]:
import math

# 1. Fonction pour convertir une requête en vecteur TF-IDF
def indexer_requete(requete, idf):
    mots = decouper_mots(requete)
    tf_requete = {}
    for mot in mots:
        mot = mot.lower()
        if mot not in tf_requete:
            tf_requete[mot] = 0
        tf_requete[mot] += 1
    total_mots = len(mots)
    for mot in tf_requete:
        tf = tf_requete[mot] / total_mots
        idf_mot = idf.get(mot, 0)
        tf_requete[mot] = tf * idf_mot
    return tf_requete

# 2. Fonction pour calculer le produit scalaire
def produit_scalaire(vec1, vec2):
    return sum(vec1.get(mot, 0) * vec2.get(mot, 0) for mot in vec1)

# 3. Fonction pour calculer la norme d'un vecteur
def norme(vec):
    return math.sqrt(sum(val**2 for val in vec.values()))

# 4. Fonction pour calculer la similarité cosinus
def calculer_sim_cosinus(vec1, vec2):
    num = produit_scalaire(vec1, vec2)
    den = norme(vec1) * norme(vec2)
    if den == 0:
        return 0.0
    return num / den

# 5. Fonction principale : calculer la similarité entre la requête et tous les documents
def calculer_ponderation_cosinus(requete, index_inverse, all_docs_tf, idf):
    requete_vec = indexer_requete(requete, idf)
    similarities = {}
    
    for doc, tf_doc in all_docs_tf.items():
        doc_vec = {mot: tf_doc[mot] * idf.get(mot, 0) for mot in tf_doc}
        sim = calculer_sim_cosinus(requete_vec, doc_vec)
        similarities[doc] = sim
    
    return similarities


#Exemple: Affichage des documents triés par similarité décroissante

In [ ]:

requete = "Commission Européenne"
similarites = calculer_ponderation_cosinus(requete, index_inverse, all_docs_tf, idf)
docs_trouves_pond_liste = sorted(similarites.items(), key=lambda x: x[1], reverse=True)
print("Top documents les plus similaires :")
for chemin, sim in docs_trouves_pond_liste[:10]:
    print(f"Sim: {sim:.4f} | {chemin.split('/')[-1]}")


Top documents les plus similaires :
Sim: 0.0012 | ep-00-01-21-fr.txt
Sim: 0.0011 | ep-00-01-18-fr.txt
Sim: 0.0011 | ep-02-12-05-fr.txt
Sim: 0.0011 | ep-03-09-25-fr.txt
Sim: 0.0010 | ep-01-12-11-fr.txt
Sim: 0.0010 | ep-00-12-13-fr.txt
Sim: 0.0010 | ep-02-10-22-fr.txt
Sim: 0.0010 | ep-02-11-18-fr.txt
Sim: 0.0010 | ep-01-04-03-fr.txt
Sim: 0.0009 | ep-01-10-23-fr.txt


# Exercice 8 : Classement par pertinence (Ranking final)
Objectif :

    - Maintenant qu’on sait calculer les similarités cosinus,
    - On veut trier proprement les documents selon leur score de similarité (ordre décroissant).
    - Affichage des 10 premiers résultats.

*On suppose que `similarites` contient {document: score} déjà calculé à l'exercice 7*

Trier les documents selon la similarité cosinus décroissante et on affiche le classement

In [40]:
docs_trouves_pond_liste = sorted(similarites.items(), key=lambda x: x[1], reverse=True)

print("Top 10 documents les plus pertinents :\n")
for chemin, sim in docs_trouves_pond_liste[:10]:
    print(f"Score: {sim:.4f} | Document: {chemin.split('/')[-1]}")

Top 10 documents les plus pertinents :

Score: 0.0012 | Document: ep-00-01-21-fr.txt
Score: 0.0011 | Document: ep-00-01-18-fr.txt
Score: 0.0011 | Document: ep-02-12-05-fr.txt
Score: 0.0011 | Document: ep-03-09-25-fr.txt
Score: 0.0010 | Document: ep-01-12-11-fr.txt
Score: 0.0010 | Document: ep-00-12-13-fr.txt
Score: 0.0010 | Document: ep-02-10-22-fr.txt
Score: 0.0010 | Document: ep-02-11-18-fr.txt
Score: 0.0010 | Document: ep-01-04-03-fr.txt
Score: 0.0009 | Document: ep-01-10-23-fr.txt


# Exercice 9 : Structuration de l’espace de recherche

Avec ce que nous avons fais, on remarque que le moteur de recherche est précis mais lent

Pourquoi ?
Parce qu’actuellement, à chaque requête, on compare la requête à TOUS les documents → lourd si on a 10 000 ou 1 000 000 documents.

Nous devons donc utiliser une méthode d'accélération comme :
    - KD-Tree
    - K-Means (clustering)
    - LSH (Locality Sensitive Hashing)

Puis comparer avec le moteur de recherche simple en terme de :
    - Temps
    - Résultats

Mon choix ce base sur l'utiliser KMeans clustering car:
 KD-Tree c'est plutôt pour la recherche sur des vecteurs, ici c’est mieux de clusteriser les documents TF-IDF

 # KMeans

Etape 1. Regrouper les documents en clusters

Importation de bibliotheque necessaire 

In [41]:
from sklearn.cluster import KMeans
import numpy as np

On convertis tous les documents en vecteurs TF-IDF (sous forme d'une matrice numpy)

In [42]:
documents = list(all_docs_tf.keys())

Création d'une matrice où chaque ligne est un document, chaque colonne un mot. Et si le mot n'est pas dans un document, on met 0

In [ ]:
vocabulaire = list(idf.keys())
doc_vectors = []

for doc in documents:
    vecteur = []
    tf_doc = all_docs_tf[doc]
    for mot in vocabulaire:
        tfidf_value = tf_doc.get(mot, 0) * idf.get(mot, 0)
        vecteur.append(tfidf_value)
    doc_vectors.append(vecteur)
    
doc_vectors = np.array(doc_vectors)
print("Matrice de documents créée :", doc_vectors.shape)


Matrice de documents créée : (363, 82144)


Etape 2. Appliquons K-Means clustering

In [ ]:
nombre_clusters = 10
kmeans = KMeans(n_clusters=nombre_clusters, random_state=42)
kmeans.fit(doc_vectors)

cluster_labels = kmeans.labels_
doc_to_cluster = {doc: cluster for doc, cluster in zip(documents, cluster_labels)}
print("Documents regroupés en clusters.")

Documents regroupés en clusters.


Etape 3. Recherche plus rapide :

Quand une requête arrive :
- On convertit la requête en vecteur TF-IDF
- On cherche seulement dans le cluster le plus proche, pas dans tous les documents.



#Exemple de recherche  rapide 

In [45]:
def recherche_clusterisee(requete, all_docs_tf, idf, doc_to_cluster, cluster_labels, doc_vectors):
    requete_vec = indexer_requete(requete, idf)
    requete_vecteur = []

    for mot in vocabulaire:
        requete_vecteur.append(requete_vec.get(mot, 0))
    requete_vecteur = np.array(requete_vecteur).reshape(1, -1)

    cluster_requete = kmeans.predict(requete_vecteur)[0]

    candidats = [doc for doc, cluster in doc_to_cluster.items() if cluster == cluster_requete]

    similarities = {}
    for doc in candidats:
        tf_doc = all_docs_tf[doc]
        doc_vec = {mot: tf_doc[mot] * idf.get(mot, 0) for mot in tf_doc}
        sim = calculer_sim_cosinus(requete_vec, doc_vec)
        similarities[doc] = sim

    return similarities

requete = "Commission Européenne"
similarites_cluster = recherche_clusterisee(requete, all_docs_tf, idf, doc_to_cluster, cluster_labels, doc_vectors)

docs_trouves_pond_liste = sorted(similarites_cluster.items(), key=lambda x: x[1], reverse=True)

print("\nTop documents trouvés dans le cluster :\n")
for chemin, sim in docs_trouves_pond_liste[:10]:
    print(f"Score: {sim:.4f} | Document: {chemin.split('/')[-1]}")



Top documents trouvés dans le cluster :

Score: 0.0012 | Document: ep-00-01-21-fr.txt
Score: 0.0011 | Document: ep-02-12-05-fr.txt
Score: 0.0011 | Document: ep-03-09-25-fr.txt
Score: 0.0010 | Document: ep-01-12-11-fr.txt
Score: 0.0010 | Document: ep-00-12-13-fr.txt
Score: 0.0010 | Document: ep-02-10-22-fr.txt
Score: 0.0010 | Document: ep-02-11-18-fr.txt
Score: 0.0010 | Document: ep-01-04-03-fr.txt
Score: 0.0009 | Document: ep-01-10-23-fr.txt
Score: 0.0009 | Document: ep-00-04-11-fr.txt


Remarque :
- La recherche est beaucoup plus rapide car on compare seulement aux documents du bon cluster.
- un bon classement par pertinence.

Avant
  - Chercher dans tous les documents 
  - Lent pour grande base     
Après
 - Chercher dans UN cluster
 - Rapide même pour grande base